In [1]:
import pandas as pd

In [2]:
# Read the data
raw_ingre_df = pd.read_csv('tmp/stat_other_ingre.csv')

In [3]:
ingre_list = raw_ingre_df[raw_ingre_df['count'] > 1]['other_ingredients'].tolist()

In [7]:
len(ingre_list)

6342

In [5]:
from googletrans import Translator

In [6]:
from googletrans import Translator
translator = Translator()

translator.translate('nước mắm', src='vi', dest='en').text

'fish sauce'

In [12]:
# index elastic search

from elasticsearch import Elasticsearch
from elasticsearch import helpers

password='FtxkVZYOjZBdsLqbryvc'
ca_certs_dir='/home/vinhvan/elasticsearch-8.4.2/config/certs/http_ca.crt'


        # Create the client instance
es = Elasticsearch(
            "https://localhost:9200",
            ca_certs=ca_certs_dir,
            basic_auth=("elastic", password),
            request_timeout=60
        )

In [13]:
es.info()

ObjectApiResponse({'name': 'VINHVAN-ITC-AI', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'nGpADS-xR_2jfOXdEXGYXw', 'version': {'number': '8.4.2', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '89f8c6d8429db93b816403ee75e5c270b43a940a', 'build_date': '2022-09-14T16:26:04.382547801Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [22]:
# create index  
index_name = 'self_match_ingre_3'
# es.indices.create(index=index_name, ignore=400)

# create mapping
setting = {
            "similarity": {
                "scripted_tfidf": {
                    "type": "scripted",
                    "script": {
                        # "source": "double tf = Math.sqrt(doc.freq); double idf = Math.log((field.docCount+1.0)/(term.docFreq+1.0)) + 1.0; double norm = 1/Math.sqrt(doc.length); return query.boost * tf * idf * norm;"
                        # this tf idf score remove idf
                        "source": "double tf = Math.sqrt(doc.freq); double idf =  1.0; double norm = 1/Math.sqrt(doc.length + 1); return query.boost * tf * idf * norm;"
                    }
                },
                "custom_bm25": {
                    "type": "BM25",
                    "b": 0.65,
                    "k1": 1.2
                }

            },
            "analysis": {
                "analyzer": {
                    "my_standard_shingle_no_unique": {
                        "tokenizer": "standard",
                        "filter": ["lowercase", "my_shingle_filter"]
                    }
                },
                "filter": {
                    "my_shingle_filter": {
                        "type": "shingle",
                        "min_shingle_size": 2,
                        "max_shingle_size": 3,
                        "output_unigrams": True
                    }
                }
            }
        }

mapping = {
    "properties": {
        "full_name_no_unique": {
            "type": "text",
            "index_phrases": True,
            "store": True,
            "analyzer": "my_standard_shingle_no_unique",
            # "similarity": "custom_bm25"
        }
    }
}

es.indices.create(index=index_name, mappings=mapping, settings=setting)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'self_match_ingre_3'})

In [23]:
resp = es.count(index=index_name, query={"match_all": {}})
print(resp)

{'count': 0, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}


In [24]:
# index into es db

for i in range(0, len(ingre_list), 1000):
    # print(i)
    # print(len(raw_dict_terms))
    print(ingre_list[i:i+1000])
    actions = [
        {
            "_index": index_name,
            "_source": {
                "full_name_no_unique": i
            }
        }
        for i in ingre_list[i:i+1000]
    ]
    helpers.bulk(es, actions)

['đường', 'muối', 'nước', 'chất điều vị', 'chất ổn định', 'chất điều chỉnh độ acid', 'chất nhũ hóa', 'bột mì', 'dầu thực vật', 'chất bảo quản', 'đậu nành', 'dầu cọ', 'tỏi', 'chất chống oxy hóa', 'sữa', 'ớt', 'chất tạo xốp', 'muối i ốt', 'chất làm dày', 'tiêu', 'maltodextrin', 'chất tạo ngọt tổng hợp', 'trứng', 'chiết xuất nấm men', 'nước mắm', 'sữa bột', 'lúa mì', 'tinh bột khoai mì', 'chất điều chỉnh độ axit', 'phẩm màu tự nhiên', 'đường tinh luyện', 'muối ăn', 'màu tự nhiên', 'hành', 'chất chống đông vón', 'gia vị', 'gạo', 'cà rốt', 'thịt heo', 'tổng hợp', 'phẩm màu tổng hợp', 'b6', 'e', 'tinh bột khoai tây', 'vắt mì', 'tinh bột bắp', 'màu thực phẩm tổng hợp', 'dầu đậu nành', 'mạch nha', 'bột ớt', 'a', 'hành tây', 'chất béo sữa', 'bơ', 'tôm', 'vitamin', 'gừng', 'd3', 'hành lá sấy', 'nước tương', 'shortening', 'bột tỏi', 'b1', 'chất béo thực vật', 'mononatri glutamat', 'chất chống oxy hoá', 'bột gạo', 'hành tím', 'bột cacao', 'cá', 'b12', 'màu tổng hợp', 'mỡ heo', 'bht', 'bột sữa', "d

In [25]:
resp = es.count(index=index_name, query={"match_all": {}})
print(resp)

{'count': 6342, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}


In [26]:
# search each term in raw_dict_terms

def get_match_term_query(raw_term, rank_list_size=10):
    j = {
        # "explain": True,
        "query": {
            "bool": {
                "should": [
                    {"match": {"full_name_no_unique": {"query": raw_term, "boost": 1}}},
                ],
                # "minimum_should_match": 1,
            }
        },
        "_source": {"includes": ["full_name_no_unique"]},
        'from': 0,
        'size': rank_list_size
    }

    return j

In [27]:
def search_term(raw_term, rank_list_size=10):
    j = get_match_term_query(raw_term, rank_list_size)
    res = es.search(index=index_name, body=j)
    return res

In [29]:
search_term(ingre_list[3])['hits']['hits']

/tmp/ipykernel_461702/777287293.py:3: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index=index_name, body=j)


[{'_index': 'self_match_ingre_3',
  '_id': '7qhY14gBE7ilYO3Ze-kM',
  '_score': 16.487139,
  '_source': {'full_name_no_unique': 'chất điều vị'}},
 {'_index': 'self_match_ingre_3',
  '_id': 'bqhY14gBE7ilYO3Ze-52',
  '_score': 15.684526,
  '_source': {'full_name_no_unique': 'chất điều vị chua'}},
 {'_index': 'self_match_ingre_3',
  '_id': 'uKhY14gBE7ilYO3Ze-93',
  '_score': 15.684526,
  '_source': {'full_name_no_unique': 'chứa chất điều vị'}},
 {'_index': 'self_match_ingre_3',
  '_id': 'wKhY14gBE7ilYO3Ze_Ly',
  '_score': 15.684526,
  '_source': {'full_name_no_unique': 'muốn chất điều vị'}},
 {'_index': 'self_match_ingre_3',
  '_id': 'EahY14gBE7ilYO3ZfPaO',
  '_score': 15.684526,
  '_source': {'full_name_no_unique': 'đường chất điều vị'}},
 {'_index': 'self_match_ingre_3',
  '_id': 'lKlY14gBE7ilYO3ZfQCQ',
  '_score': 15.684526,
  '_source': {'full_name_no_unique': 'chất điều vị gồm'}},
 {'_index': 'self_match_ingre_3',
  '_id': 'CahY14gBE7ilYO3Ze-1A',
  '_score': 14.962142,
  '_source': {'

In [30]:
# match term
ret_list = []
count_done = 0
for term in ingre_list:
    res = search_term(term, 10)
    ret_list.append(res)

    count_done = count_done + 1
    # print(count)
    # if count_done % 1000 == 0:
    #     print("{}/{}".format(count_done, len(raw_dict_terms)))

/tmp/ipykernel_461702/777287293.py:3: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index=index_name, body=j)


In [31]:
def extract_name_and_score(res):
    return [(i['_source']['full_name_no_unique'], i['_score']) for i in res['hits']['hits']]

In [32]:
clean_pair_list = [extract_name_and_score(i) for i in ret_list]

In [34]:
query_result_pair = list(zip(ingre_list, clean_pair_list))

In [40]:
# query_result_pair

In [41]:
# extract match pair
match_pair_list = list()
for query, results in query_result_pair:
    for result in results:
        if query == result[0]:
            continue
        else:
            match_pair_list.append({
                "query": query, 
                "match": result[0],
                 "score": result[1]})


In [43]:

match_df = pd.DataFrame(match_pair_list)
match_df.to_csv('tmp/ingre_match_pair.csv', index=False)